In [1]:
%matplotlib inline

## 一个神经网络的典型训练过程如下：
- 定义包含可学习参数的神经网络模型
- 在数据集上迭代
- 通过神经网络处理输入
- 计算损失函数
- 计算损失函数的梯度反向传播回去的参数
- 更新网络参数

In [2]:
# 定义一个网络
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):  
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel
        # 6 output image channel
        # 5*5 square convolution
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # affine operation:
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
        

- 模型中必须定义forward函数， backward函数会被autograd自动创建。
- net.parameters()返回可以被学习的参数。


In [4]:
net = Net()
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [5]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)


tensor([[ 0.1057, -0.1054,  0.1059,  0.0558, -0.1155, -0.0152, -0.1152, -0.1473,
         -0.1676,  0.0847]], grad_fn=<AddmmBackward0>)


## 损失函数：

In [8]:
criterion = nn.MSELoss()
out.shape

torch.Size([1, 10])

In [9]:
target = torch.randn(1,10)
loss = criterion(out,target)
print(loss)
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

tensor(1.9393, grad_fn=<MseLossBackward0>)


## 反向传播
- 调用loss.backward()获得反向传播的误差。
- 但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度。
- 现在，我们将调用loss.backward()，并查看conv1层的偏差（bias）项在反向传播前后的梯度。

In [10]:
net.zero_grad()

In [11]:
loss.backward()

## 更新权重

In [12]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [13]:
## 使用torch.optim包实现所有的不同学习率的更新规则：
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
## when in training loop:
optimizer.zero_grad()
input = torch.randn(1,1,32,32)
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()
